<a href="https://colab.research.google.com/github/geonextgis/Mastering-Machine-Learning-and-GEE-for-Earth-Science/blob/main/00_geemap/01_Cloud_Masking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cloud Masking**

## **Import the Required Libraries**

In [76]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [77]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import ee
import geemap

## **Initialize a Map**

In [78]:
# # Trigger the authentication flow.
# ee.Authenticate()

# # Initialize the library.
# ee.Initialize(project='my-project')

In [79]:
Map = geemap.Map(height="400pt")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## **Define a Region of Interest**

In [80]:
# Read the shapefile of the West Bengal state using geopandas
shp_path = r"/content/drive/MyDrive/Colab Notebooks/GitHub Repo/Mastering-Machine-Learning-and-GEE-for-Earth-Science/Datasets/West_Bengal_Boundary/District_shape_West_Bengal.shp"
wb_gdf = gpd.read_file(shp_path)
print(wb_gdf.shape)
wb_gdf.head()

(24, 5)


,GM_LAYER,NAME,NAME_0,NAME_1,geometry
0,Unknown Area Type,Jhargram,India,West Bengal,"POLYGON ((87.07253 22.72494, 87.07639 22.72033..."
1,Unknown Area Type,Kalimpong,India,West Bengal,"POLYGON ((88.43195 27.08161, 88.43962 27.08417..."
2,Unknown Area Type,Kolkata,India,West Bengal,"POLYGON ((88.40731 22.55647, 88.40570 22.55393..."
3,Unknown Area Type,Paschim Barddhaman,India,West Bengal,"POLYGON ((87.57518 23.52770, 87.57636 23.52456..."
4,Unknown Area Type,Pashchim Medinipur,India,West Bengal,"POLYGON ((87.88528 22.52200, 87.88857 22.51518..."


In [81]:
# Filter the 'Bankura' district from the geodataframe
roi_gdf = wb_gdf[wb_gdf['NAME']=="Bankura"]
roi_gdf

,GM_LAYER,NAME,NAME_0,NAME_1,geometry
6,Unknown Area Type,Bankura,India,West Bengal,"POLYGON ((86.89942 23.63156, 86.91116 23.62734..."


In [82]:
# Push the filtered geometry to the Earth Engine
roi_ee = geemap.gdf_to_ee(roi_gdf)

vis_params = {
    "fillColor": "00000000",
    "color": "black",
    "width": 1
}
Map.addLayer(roi_ee.style(**vis_params), {}, "ROI")
Map.centerObject(roi_ee, 9)

## **Cloud Masking on Landsat Data**

### **Filtering Landsat 9 Image Collection**

In [83]:
# Read Landsat 9 image collection from EE
L9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")

# Filter the image collection with roi, daterange, and cloud cover property
L9Filtered = L9.filterBounds(roi_ee)\
               .filterDate("2022-01-01", "2022-12-31")\
               .filterMetadata("CLOUD_COVER", "less_than", 50)

# Print the size of the filtered image collection
L9Filtered.size().getInfo()

27

### **Function to Remove Clouds and Shadows**

In [84]:
# Write a function to remove clouds from Landsat 9 imagery
def maskL9CloudsAndShadows(image):

    # Read the 'QA_PIXEL' (Quality Assessment) band
    qa = image.select("QA_PIXEL")

    # Define all the variables
    dilated_cloud_bitmask = 1 << 1
    cirrus_bitmask = 1 << 2
    cloud_bitmask = 1 << 3
    cloud_shadow_bitmask = 1 << 4

    # Create a mask
    mask = qa.bitwiseAnd(dilated_cloud_bitmask).eq(0).And(
           qa.bitwiseAnd(cirrus_bitmask).eq(0)).And(
           qa.bitwiseAnd(cloud_bitmask).eq(0)).And(
           qa.bitwiseAnd(cloud_shadow_bitmask).eq(0))

    return image.updateMask(mask)

🤔 **Note:** <br>
1. **`bitwiseAnd` Function:**<br>
The `bitwiseAnd` function is used to perform a bitwise `AND` operation on the bits of two numbers or image bands. It takes two operands and returns a result where each bit position in the output is the logical AND of the corresponding bits in the input operands.

2. **`eq` Function:**<br>
The `eq` function is used for element-wise equality comparison. It returns a binary image where each pixel is set to 1 if the corresponding pixels in the input images are equal and 0 otherwise.

### **Implementation on an Image and Image Collection**

In [85]:
# Check the cloud cover value of the first image
# L9Filtered.first()

In [86]:
# Display the first image of the filtered image collection
L9FilteredFirst = L9Filtered.first()

# Display a Standard False Color Composite (SFCC)
sfcc_vis = {
    "min": 8000,
    "max": 17000,
    "bands": ["SR_B5", "SR_B4", "SR_B3"]
}
Map.addLayer(L9FilteredFirst, sfcc_vis, "Landsat 9 Image")

In [87]:
# Apply the 'maskL9CloudsAndShadows' function on the first image of the filtered image collection
cloud_free_image = maskL9CloudsAndShadows(L9FilteredFirst)
Map.addLayer(cloud_free_image, sfcc_vis, "Landsat 9 Cloud Free Image")

In [88]:
# Apply the 'maskL9CloudsAndShadows' function on the filtered image collection
# to create a cloud free composite image of the whole year
cloud_free_composite = L9Filtered.map(maskL9CloudsAndShadows)\
                                 .median()\
                                 .clip(roi_ee)
Map.addLayer(cloud_free_composite, sfcc_vis, "Landsat 9 Cloud Free Composite")

🤔 **Note:** The `map` function in GEE is commonly used for applying a specified function to each element of a collection. This function is particularly useful for processing each image in an image collection or each feature in a feature collection.